In [ ]:
import os
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("Api key found. Good to go!")   
else:
    print("No correct api key was found")
MODEL = "gpt-4o-mini"
openai = OpenAI(api_key=api_key)

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
class GithubProfile:

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]
            
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
profile = GithubProfile("https://github.com/ertgl")
profile.links

In [ ]:
link_system_prompt = "You are provided with a list of links found on a Github page. \
You are able to decide which of the links would be most relevant to include in a portfolio about the github user, \
such as links to an About page, or a Repositories, or Projects.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "overview page", "url": "https://another.full.url"},
        {"type": "repositories page": "url": "https://another.full.url?tab=repositories"}
    ]
}
"""

In [ ]:
print(link_system_prompt)


In [ ]:
def get_links_user_prompt(profile):
    user_prompt = f"Here is the list of links on the website of {profile.url} - "
    user_prompt += "please decide which of these are relevant web links for a portfolio about the user, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, Login, Blog or Github trending related pages.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(profile.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(profile))

In [ ]:
def get_links(url):
    profile = GithubProfile(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(profile)}
        ],
        response_format= {"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
get_links(profile.url)

# Second step: make the portfolio!

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += GithubProfile(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += GithubProfile(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://github.com/ertgl"))

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a personal github page \
and creates a short portfolio about the user profile, especially projects and repositories and summary of the repo's \
README files for prospective recruiters, investors. Respond in markdown.\
Include details of person profile overview, if you have the information."

In [ ]:
def get_portfolio_user_prompt(profile_name, url):
    user_prompt = f"You are looking at a user called: {profile_name} on Github.\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short portfolio of the user in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
print(get_portfolio_user_prompt("Ertuğrul Noyan Keremoğlu", "https://github.com/ertgl"))


In [ ]:
def create_portfolio(profile_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_portfolio_user_prompt(profile_name, url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_portfolio("Ertuğrul Noyan Keremoğlu", "https://github.com/ertgl")

In [ ]:
def stream_portfolio(profile_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_portfolio_user_prompt(profile_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_portfolio("Ertuğrul Noyan Keremoğlu", "https://github.com/ertgl")

# Multi-lingual with Multi-Tone in Desire Format

In [100]:
def multi_lingual_stream_portfolio(profile_name, url, language, tone):

    system_prompt = f"""
You are an assistant that analyzes the contents of several relevant pages from a github profile page and 
creates a visually appealing and professional short portfolio for prospective investors, and recruiters. 
The portfolio should be written in {language} and use a {tone.lower()} tone throughout.

The portfolio should follow this structure (in {language}):

1. **Front Cover**:
   - Prominently display the user name as Title.
   - Include a compelling headline or tagline.
   - Add something engaging relevant to the user’s summarized README files if available.

2. **About**:
   - Provide a brief introduction to the user's projects approach.
   - State which repository they own or they contributed.

3. **Overview**:
   - Summarize the user's projects, repositories, or solutions by summarized README files if available.
   - Highlight benefits or unique developer/development points.
   - Mention the follower and following users count and total stars they got.


4. **My Culture**:
   - Outline the user’s key values or guiding principles.
   - Describe the workplace environment (e.g., innovation-driven, inclusive, collaborative).
   - Highlight community engagement.

5. **What kind of companies may be interested**:
   - Describe the target customers or industries served.
   - Mention open source contributions also if available. 
   
6. **Projects**:
    
   ***Owner***:
   - List owned projects/repositories with summaries. (Summarize README file of the each project)
   
   ***Contributer***:
   - List contributed projects/repositories with summaries. (Summarize README file of the each project)


7. **Support and Donation**:
   - Encourage those interested in user's open source projects to donate.
   - Provide direct links or steps to apply if available.

8. **Contact Us**:
   - Provide the user’s address, phone number, and email.
   - Include links to social media platforms.
   - Add a link to the user’s website.

9. **Closing Note**:
   - End with a thank-you message or an inspirational note for the reader.
   - Add a call-to-action (e.g., “Get in touch today!” or “Explore more on my website”).

Ensure the content is concise, engaging, visually clear, and tailored to the target audience. Use headings and subheadings to make the brochure easy to navigate. Include links and contact information wherever applicable.
"""


    
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_portfolio_user_prompt(profile_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
multi_lingual_stream_portfolio("Ertuğrul Noyan Keremoğlu", "https://github.com/ertgl", "English", "serious, entertaining, witty")